## Notebook exemplificando o uso das funcionalidades do YOLO-MS
Representação do fluxo de trabalho, desde o treinamento até a inferência.

In [26]:
import cv2
import json
from pathlib import Path
from sklearn.model_selection import train_test_split

def convert_to_coco(dataset: str, output_dir: str, train_ratio: float = 0.6, val_ratio: float = 0.2):
    """
    Converte dataset pedrozamboni para formato COCO com divisão 60/20/20.
    """

    dataset_dir = Path(dataset)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Cria estrutura do formato COCO com pasta de teste
    (output_dir / "images" / "train").mkdir(parents=True, exist_ok=True)
    (output_dir / "images" / "val").mkdir(parents=True, exist_ok=True)
    (output_dir / "images" / "test").mkdir(parents=True, exist_ok=True)
    (output_dir / "annotations").mkdir(parents=True, exist_ok=True)

    # Obtém arquivos de anotação
    bbox_files = list((dataset_dir / "bbox_txt").glob("*.txt"))
    image_files = list((dataset_dir / "images").glob("*.png"))

    matched_data = []
    for bbox_file in bbox_files:
        img_id = bbox_file.stem
        img_file = None

        for img_file in image_files:
            if img_file.stem == img_id:
                img_file = img_file
                break

        if img_file and img_file.exists():
            matched_data.append((img_file, bbox_file))

    print(f"Encontrados {len(matched_data)} arquivos de imagem e bbox correspondentes.")
    
    # Primeira divisão: 60% treino, 40% temporário (para val + teste)
    train_data, temp_data = train_test_split(
        matched_data, 
        train_size=train_ratio, 
        random_state=42
    )
    
    # Segunda divisão: 20% val, 20% teste dos 40% restantes
    val_ratio_adjusted = val_ratio / (1 - train_ratio)  # 0.2 / 0.4 = 0.5
    val_data, test_data = train_test_split(
        temp_data, 
        train_size=val_ratio_adjusted, 
        random_state=42
    )

    print(f"Divisão dos dados:")
    print(f"  Treino: {len(train_data)} imagens ({len(train_data)/len(matched_data)*100:.1f}%)")
    print(f"  Val:    {len(val_data)} imagens ({len(val_data)/len(matched_data)*100:.1f}%)")
    print(f"  Teste:  {len(test_data)} imagens ({len(test_data)/len(matched_data)*100:.1f}%)")

    # Processa datasets de treino, validação e teste
    for split, data in [("train", train_data), ("val", val_data), ("test", test_data)]:
        images = []
        annotations = []
        annotation_id = 1

        for img_id, (img_file, bbox_file) in enumerate(data):
            # Lê imagem
            img = cv2.imread(str(img_file))
            if img is None:
                print(f"Aviso: Não foi possível ler a imagem {img_file}. Pulando.")
                continue

            height, width = img.shape[:2]

            # Copia imagem para diretório de saída
            output_img_path = output_dir / "images" / split / img_file.name
            cv2.imwrite(str(output_img_path), img)

            images.append({
                "id": img_id,
                "file_name": img_file.name,
                "width": width,
                "height": height
            })

            # Lê bounding boxes
            with open(bbox_file, "r") as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue

                    parts = line.split() 
                    if len(parts) == 4:
                        x1, y1, x2, y2 = map(float, parts)

                        # Converte para formato COCO (x, y, largura, altura)
                        x = min(x1, x2)
                        y = min(y1, y2)
                        bbox_width = abs(x2 - x1)
                        bbox_height = abs(y2 - y1)
                        
                        # Garante que as coordenadas estão dentro dos limites da imagem
                        x = max(0, min(x, width - 1))
                        y = max(0, min(y, height - 1))
                        bbox_width = min(bbox_width, width - x)
                        bbox_height = min(bbox_height, height - y)

                        if bbox_width > 0 and bbox_height > 0:
                            annotations.append({
                                "id": annotation_id,
                                "image_id": img_id,
                                "category_id": 1,  # Assumindo uma única categoria
                                "bbox": [x, y, bbox_width, bbox_height],
                                "area": bbox_width * bbox_height,
                                "iscrowd": 0
                            })
                            annotation_id += 1
                    else:
                        print(f"Aviso: Formato de bbox inesperado em {bbox_file}. Esperado 4 valores, obtido {len(parts)}. Pulando linha: {line}")
        
        # Cria formato COCO
        coco_format = {
            "images": images,
            "annotations": annotations,
            "categories": [
                {
                    "id": 1,
                    "name": "tree",
                    "supercategory": "plant"
                }
            ]
        }
        
        # Salva arquivo de anotação
        with open(output_dir / "annotations" / f"instances_{split}.json", 'w') as f:
            json.dump(coco_format, f, indent=2)
        
        print(f"✅ {split}: {len(images)} imagens, {len(annotations)} anotações")

In [31]:
convert_to_coco(
    dataset="pedrozamboni_dataset",
    output_dir="coco_dataset",
    train_ratio=0.6,  # 60% train
    val_ratio=0.2     # 20% val
)

Encontrados 220 arquivos de imagem e bbox correspondentes.
Divisão dos dados:
  Treino: 132 imagens (60.0%)
  Val:    44 imagens (20.0%)
  Teste:  44 imagens (20.0%)
✅ train: 132 imagens, 2014 anotações
✅ train: 132 imagens, 2014 anotações
✅ val: 44 imagens, 707 anotações
✅ val: 44 imagens, 707 anotações
✅ test: 44 imagens, 632 anotações
✅ test: 44 imagens, 632 anotações


## Criação de um config file para o YOLO-MS Fine-tune
Ajuste de hiperparâmetros e configuração do modelo YOLO-MS para o treinamento. configuração apenas de uma única classe ('tree') para o fine-tune do modelo YOLO-MS de 23M de parâmetros.

In [32]:
#_base_ = 'yoloms_syncbn_fast_8xb32-300e_coco.py'
_base_ = 'mmyolo/configs/yoloms/yoloms_syncbn_fast_8xb32-300e_coco.py'
# Configuração do dataset
data_root = 'D:/UnB/IIA/YOLO-MS-IIA/coco_dataset'
class_name = ('tree',)  # única classe
num_classes = 1
metainfo = dict(classes=class_name, palette=[(0, 255, 0)])

# Parâmetros de treinamento para fine-tuning
max_epochs = 300  # Reduzido de 300 para fine-tuning
train_batch_size_per_gpu = 2
val_batch_size_per_gpu = 1
train_num_workers = 2
val_num_workers = 1

# Taxa de aprendizado menor para fine-tuning
base_lr = 0.0001


# Configuração do modelo - Atualizar para classe única
model = dict(
    bbox_head=dict(
        head_module=dict(num_classes=num_classes)
    ),
    train_cfg=dict(
        assigner=dict(num_classes=num_classes)
    )
)

# Configuração dos dados
train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        ann_file='annotations/instances_train.json',
        data_prefix=dict(img='images/train/'),
        metainfo=metainfo
    )
)

val_dataloader = dict(
    batch_size=val_batch_size_per_gpu,
    num_workers=val_num_workers,
    dataset=dict(
        data_root=data_root,
        ann_file='annotations/instances_val.json',
        data_prefix=dict(img='images/val/'),
        metainfo=metainfo,
        test_mode=True
    )
)

# Adicionar dataloader de teste
test_dataloader = dict(
    batch_size=val_batch_size_per_gpu,
    num_workers=val_num_workers,
    dataset=dict(
        data_root=data_root,
        ann_file='annotations/instances_test.json',
        data_prefix=dict(img='images/test/'),
        metainfo=metainfo,
        test_mode=True
    )
)

# Avaliadores
val_evaluator = dict(
    type='mmdet.CocoMetric',
    ann_file=f'{data_root}/annotations/instances_val.json',
    metric='bbox',
    format_only=False,
    classwise=True,
)

test_evaluator = dict(
    type='mmdet.CocoMetric',
    ann_file=f'{data_root}/annotations/instances_test.json',
    metric='bbox',
    format_only=False,
    classwise=True,
)

# Configuração de treinamento
train_cfg = dict(
    type='EpochBasedTrainLoop',
    max_epochs=max_epochs,
    val_interval=20
)

val_cfg = dict(type='ValLoop')
test_cfg = dict(type='TestLoop')
    
# Escalonamento automático da taxa de aprendizado
auto_scale_lr = dict(enable=True, base_batch_size=16)

# Configuração de hooks
default_hooks = dict(
    checkpoint=dict(
        type='CheckpointHook',
        interval=20,
        save_best='coco/bbox_mAP',
        rule='greater',
        max_keep_ckpts=5
    ),
    logger=dict(type='LoggerHook', interval=20)
)

# Carregar pesos pré-treinados para fine-tuning
load_from = 'D:/UnB/IIA/YOLO-MS-IIA/pre-trained.pth'

Esse código está salvo no arquivo `yoloms_trees_finetune.py` e pode ser utilizado para treinar o modelo YOLO-MS com os dados do dataset convertido para o formato COCO.

## Fine-tune do modelo
Como o dataset COCO e o arquivo de configuração do modelo, o próximo passo é realizar o treinamento de pesos do modelo YOLO-MS com os dados do dataset convertido.

In [ ]:
!python mmyolo/tools/train.py "D:/UnB/IIA/YOLO-MS-IIA/yoloms_trees_finetune.py" --work-dir "D:/UnB/IIA/YOLO-MS-IIA/finetune"

------------------------------------------------------------
System environment:
    sys.platform: win32
    Python: 3.8.20 (default, Oct  3 2024, 15:19:54) [MSC v.1929 64 bit (AMD64)]
    CUDA available: True
    numpy_random_seed: 633647791
    GPU 0: NVIDIA GeForce RTX 2060 SUPER
    CUDA_HOME: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.6
    NVCC: Cuda compilation tools, release 12.6, V12.6.20
    GCC: n/a
    PyTorch: 1.12.1
    PyTorch compiling details: PyTorch built with:
  - C++ Version: 199711
  - MSVC 192829337
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 2019
  - LAPACK is enabled (usually provided by MKL)
  - CPU capability usage: AVX2
  - CUDA Runtime 11.3
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;ar

## Teste do modelo treinado

In [24]:
!python mmyolo/demo/image_demo.py "finetune/one-stage.png" "yoloms_trees_finetune.py" "finetune/best_coco/best_epoch_200.pth" --show

Loads checkpoint by local backend from path: finetune/best_coco/best_epoch_200.pth
07/01 21:06:57 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
[                                                  ] 0/1, elapsed: 0s, ETA:07/01 21:06:57 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
07/01 21:06:57 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1/1, 0.3 task/s, elapsed: 3s, ETA:     0s


e:\Anaconda\envs\yolo\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
e:\Anaconda\envs\yolo\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
e:\Anaconda\envs\yolo\lib\site-packages\mmengine\visualization\visualizer.py:741: UserWarning: Warning: The bbox is out of bounds, the drawn bbox may not be in the image
  warnings.warn(
e:\Anaconda\envs\yolo\lib\site-packages\mmengine\visualization\visualizer.py:812: UserWarning: Warning: The polygon is out of bounds, the drawn polygon

![one-stage-yoloms](finetune/one-stage-yoloms.png)

In [25]:
!python mmyolo/demo/image_demo.py "finetune/two-stage.png" "yoloms_trees_finetune.py" "finetune/best_coco/best_epoch_200.pth" --show

Loads checkpoint by local backend from path: finetune/best_coco/best_epoch_200.pth
07/01 21:07:06 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.
[                                                  ] 0/1, elapsed: 0s, ETA:07/01 21:07:06 - mmengine - WARNING - "FileClient" will be deprecated in future. Please use io functions in https://mmengine.readthedocs.io/en/latest/api/fileio.html#file-io
07/01 21:07:06 - mmengine - WARNING - "HardDiskBackend" is the alias of "LocalBackend" and the former will be deprecated in future.

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1/1, 0.4 task/s, elapsed: 3s, ETA:     0s


e:\Anaconda\envs\yolo\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
e:\Anaconda\envs\yolo\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
e:\Anaconda\envs\yolo\lib\site-packages\mmengine\visualization\visualizer.py:741: UserWarning: Warning: The bbox is out of bounds, the drawn bbox may not be in the image
  warnings.warn(
e:\Anaconda\envs\yolo\lib\site-packages\mmengine\visualization\visualizer.py:812: UserWarning: Warning: The polygon is out of bounds, the drawn polygon

![two-stage-yoloms](finetune/two-stage-yoloms.png)

## Gráfico de perda até a epoca 25
![loss-epoch-25](grafico.png)

## Tabela comparativa entre os resultados dos diferentes modelos testados no experimento

# Resultados

| Model                                              | Test Set AP50 |
| :------------------------------------------------- | :-----------: |
| **Anchor-Free (AF) Methods**                       |               |
| FSAF                                               |     0.701     |
| ATSS                                               |     0.692     |
| FoveaBox                                           |     0.692     |
| VarifocalNet (2)                                   |     0.683     |
| VarifocalNet (1)                                   |     0.664     |
| **One-Stage Methods**                              |               |
| YOLO-MS (Epoch 200)                                |     0.748     |
| YOLO-MS (Epoch 300)                                |     0.739     |
| Gradient Harmonized Single-stage Detector          |     0.691     |
| Generalized Focal Loss                             |     0.677     |
| Probabilistic Anchor Assignment                    |     0.677     |
| SABL                                               |     0.661     |
| NAS-FPN                                            |     0.658     |
| RetinaNet                                          |     0.650     |
| YoloV3                                             |     0.591     |
| **Two-Stage and Multi-Stage (DetectorRS) Methods** |               |
| Double Heads                                       |     0.699     |
| CARAFE                                             |     0.697     |
| Empirical Attention                                |     0.690     |
| Mixed precision training                           |     0.679     |
| Faster R-CNN                                       |     0.660     |
| Deformable ConvNets v2                             |     0.657     |
| Dynamic R-CNN                                      |     0.655     |
| DetecorRS                                          |     0.651     |
| Weight Standardization                             |     0.631     |

# Comparação dos resultados do modelo YOLO-MS treinado com diferentes épocas

| model     |  mAP  | mAP_50 | mAP_75 | mAP_s | mAP_m | mAP_l |
| :-------- | :---: | :----: | :----: | :---: | :---: | :---: |
| Epoch 300 | 0.405 | 0.739  | 0.403  | 0.227 | 0.449 | 0.653 |
| Epoch 200 | 0.416 | 0.748  | 0.421  | 0.246 | 0.463 | 0.616 |

Diferenças utilizadas nesse projeto

- arquivo de config para fine tune
- script para conversão para dataset coco